# Otto challenge main notebook

In [1]:
# download dataset

%%shell
if ! [ -d "dataset" ]; then
  curl -L -o 'dataset.zip' 'https://docs.google.com/uc?export=download&id=1_F09b-A1xdAjOXKE7STqgiauSu4CnL1p'
  unzip dataset.zip
fi;

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0   1048      0 --:--:-- --:--:-- --:--:--  1048
100 5510k    0 5510k    0     0  9019k      0 --:--:-- --:--:-- --:--:-- 9019k
Archive:  dataset.zip
   creating: dataset/
  inflating: dataset/test.csv        
  inflating: dataset/train.csv       


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import time

In [0]:
# load data
train = pd.read_csv('dataset/train.csv');
test = pd.read_csv('dataset/test.csv');

In [4]:
train.head()

,id,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,feat_11,feat_12,feat_13,feat_14,feat_15,feat_16,feat_17,feat_18,feat_19,feat_20,feat_21,feat_22,feat_23,feat_24,feat_25,feat_26,feat_27,feat_28,feat_29,feat_30,feat_31,feat_32,feat_33,feat_34,feat_35,feat_36,feat_37,feat_38,feat_39,...,feat_55,feat_56,feat_57,feat_58,feat_59,feat_60,feat_61,feat_62,feat_63,feat_64,feat_65,feat_66,feat_67,feat_68,feat_69,feat_70,feat_71,feat_72,feat_73,feat_74,feat_75,feat_76,feat_77,feat_78,feat_79,feat_80,feat_81,feat_82,feat_83,feat_84,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,target
0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2,0,0,0,0,1,0,4,1,1,0,0,2,0,0,0,0,0,1,0,0,0,0,...,0,0,2,0,0,11,0,1,1,0,1,0,7,0,0,0,1,0,0,0,0,0,0,0,2,1,0,0,0,0,1,0,0,0,0,0,0,0,0,Class_1
1,2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,2,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Class_1
2,3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,6,0,0,2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,Class_1
3,4,1,0,0,1,6,1,5,0,0,1,1,0,1,0,0,1,1,0,0,0,0,0,0,7,2,2,0,0,0,58,0,10,0,0,0,0,0,3,0,...,1,0,0,0,0,0,0,0,0,0,2,1,5,0,0,4,0,0,2,1,0,1,0,0,1,1,2,2,0,22,0,1,2,0,0,0,0,0,0,Class_1
4,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,3,0,0,0,0,0,0,0,4,0,1,0,0,0,1,0,0,0,0,1,0,0,0,Class_1


## Pre-processing

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# preprocessing
def pre_processing(data):

  # 1) Separate target from features
  y = data['target']
  # drop 'target' and 'id' columns
  X = data.drop(['target'], axis=1)
  X = X.drop(['id'], axis=1)

  # 2) Label encoding
  le = LabelEncoder()
  y = le.fit_transform(y)
  y = pd.DataFrame(y)
  
  # Eliminate repeated rows

  return X, y



# train/test split
def split(X, y, train_perc = 0.8):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 1 - train_perc)
  return X_train, X_test, y_train, y_test 

In [0]:
X, y = pre_processing(train)

# check distributions

plt.hist(y_train)
plt.hist(y_test)
plt.show()


X_train, X_test, y_train, y_test = split(X, y)

## Feature Engineering

### LightGBM

In [0]:
import lightgbm

In [67]:
parameters = {
    #'application': 'sigmoid',
    'objective': 'multiclass',
    'metric': 'multi_logloss',
    'is_unbalance': 'true',
    'num_class':9,
    'boosting': 'gbdt',
    'num_leaves': 31,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'learning_rate': 0.05,
    'verbose': 0
}

train_data = lightgbm.Dataset(X_train, label=y_train)
test_data = lightgbm.Dataset(X_test, label=y_test)

model = lightgbm.train(parameters,
                       train_data,
                       valid_sets=test_data,
                       num_boost_round=5000,
                       early_stopping_rounds=100)

[1]	valid_0's multi_logloss: 1.88008
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's multi_logloss: 1.80793
[3]	valid_0's multi_logloss: 1.7448
[4]	valid_0's multi_logloss: 1.68435
[5]	valid_0's multi_logloss: 1.63748
[6]	valid_0's multi_logloss: 1.59177
[7]	valid_0's multi_logloss: 1.55047
[8]	valid_0's multi_logloss: 1.50504
[9]	valid_0's multi_logloss: 1.46613
[10]	valid_0's multi_logloss: 1.43087
[11]	valid_0's multi_logloss: 1.39537
[12]	valid_0's multi_logloss: 1.36377
[13]	valid_0's multi_logloss: 1.33413
[14]	valid_0's multi_logloss: 1.30795
[15]	valid_0's multi_logloss: 1.27983
[16]	valid_0's multi_logloss: 1.25506
[17]	valid_0's multi_logloss: 1.23317
[18]	valid_0's multi_logloss: 1.21212
[19]	valid_0's multi_logloss: 1.18925
[20]	valid_0's multi_logloss: 1.16655
[21]	valid_0's multi_logloss: 1.14713
[22]	valid_0's multi_logloss: 1.12674
[23]	valid_0's multi_logloss: 1.10911
[24]	valid_0's multi_logloss: 1.09148
[25]	valid_0's multi_logloss: 1.073